In [33]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
if project_root not in sys.path:
    sys.path.append(project_root)

In [34]:
from tbparse import SummaryReader
import pandas as pd

In [35]:
log_dir = "../logs/tensorboard/"
reader = SummaryReader(log_dir, extra_columns={"dir_name"})
df = reader.scalars

In [36]:
df.head()

,step,tag,value,dir_name
0,0,accuracy/val,0.750100,cnn/grayscale/log_0
1,1,accuracy/val,0.778518,cnn/grayscale/log_0
2,2,accuracy/val,0.805336,cnn/grayscale/log_0
3,3,accuracy/val,0.824583,cnn/grayscale/log_0
4,4,accuracy/val,0.803700,cnn/grayscale/log_0


In [37]:
df[["model", "representation", "log"]] = df["dir_name"].str.split("/", expand=True)
df[["metric", "data_set"]] = df.tag.str.split("/", expand=True)
df.drop(["dir_name", "tag"], axis=1, inplace=True)

In [38]:
df.head()

,step,value,model,representation,log,metric,data_set
0,0,0.750100,cnn,grayscale,log_0,accuracy,val
1,1,0.778518,cnn,grayscale,log_0,accuracy,val
2,2,0.805336,cnn,grayscale,log_0,accuracy,val
3,3,0.824583,cnn,grayscale,log_0,accuracy,val
4,4,0.803700,cnn,grayscale,log_0,accuracy,val


In [39]:
loss = df.query("metric == 'loss' and data_set == 'val'")

In [40]:
loss.head()

,step,value,model,representation,log,metric,data_set
90,0,24.989994,cnn,grayscale,log_0,loss,val
91,1,22.148165,cnn,grayscale,log_0,loss,val
92,2,19.466440,cnn,grayscale,log_0,loss,val
93,3,17.541723,cnn,grayscale,log_0,loss,val
94,4,19.630022,cnn,grayscale,log_0,loss,val


# Validation loss

In [41]:
loss.groupby(["model", "representation"])["value"].min().round(decimals=2).to_frame().T

model                cnn logistic_regression            lstm          \
representation grayscale             huffman refined huffman refined   
value              15.33               34.41   29.85   34.56   30.24   

model          transformer          
representation     huffman refined  
value                34.56   29.43

# Validation Metrics

In [42]:
metrics = df.query("metric != 'loss'")

In [43]:
metrics.groupby(["metric", "model", "representation"])["value"].max().round(
    decimals=2
).to_frame()

value
metric    model               representation       
accuracy  cnn                 grayscale        0.85
          logistic_regression huffman          0.66
                              refined          0.70
          lstm                huffman          0.65
                              refined          0.70
          transformer         huffman          0.65
                              refined          0.71
f1        cnn                 grayscale        0.86
          logistic_regression huffman          0.73
                              refined          0.75
          lstm                huffman          0.74
                              refined          0.76
          transformer         huffman          0.72
                              refined          0.76
precision cnn                 grayscale        0.87
          logistic_regression huffman          0.66
                              refined          0.72
          lstm                huffman          0.65
                              refined          0.70
          transformer         huffman          0.65
                              refined          0.69
recall    cnn                 grayscale        0.92
          logistic_regression huffman          0.89
                              refined          0.88
          lstm                huffman          0.92
                              refined          0.92
          transformer         huffman          0.81
                              refined          0.97

# Best Checkpoints

In [44]:
loss.loc[loss.groupby(["model", "representation"])["value"].idxmin()][["model", "representation", "log"]]

,model,representation,log
118,cnn,grayscale,log_0
703,logistic_regression,huffman,log_0
1338,logistic_regression,refined,log_0
1721,lstm,huffman,log_0
2661,lstm,refined,log_2
2787,transformer,huffman,log_0
3211,transformer,refined,log_1
